# Optimizing the Autoregressive model

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Loading TensorBoard
%load_ext tensorboard

In [3]:
from tensorboard.plugins.hparams import api as hp

In [4]:
import random
random.seed(123)

In [5]:
# import window_generator as WindowGenerator
from window_generator import WindowGenerator

# Configuring hyperparameters

In [6]:
# Optimizers (SGD, Adam, Adamax, RMSprop)
optimizers = [
    'adam', 
    'sgd', 
    'adamax', 
    'rmsprop'
]


# Loss functions (MeanSquaredError, Huber, MeanSquaredLogarithmicError)
loss_functions = [
    'huber', 
    'MeanSquaredError', 
    'MeanSquaredLogarithmicError'
]

In [7]:
# Hyperparameters
LSTM_UNITS = hp.HParam('LSTM_units', hp.Discrete([16, 32, 64, 128]))
OPTIMIZER = hp.HParam('Optimizer', hp.Discrete(optimizers))
LOSS_FUNCTION = hp.HParam('Loss_function', hp.Discrete(loss_functions))
WINDOW_SIZE = hp.HParam('WINDOW_SIZE', hp.Discrete([12, 24, 48]))

# Metrics
METRIC_RMSE = 'RootMeanSquaredError'


with tf.summary.create_file_writer('logs/AR').as_default():
    hp.hparams_config(
    # Parameters
    hparams = [
        LSTM_UNITS,
        OPTIMIZER,
        WINDOW_SIZE,
        LOSS_FUNCTION    
    ],
    # Metrics
    metrics = [
        hp.Metric(
            METRIC_RMSE, display_name='RMSE'
        ),
    ]
    )

# Loading the dataset

In [8]:
df = pd.read_csv('extracted_data/csv/hour_merged.csv',
                 parse_dates={'data': ['date', 'time']},
                 infer_datetime_format=True,
                 dayfirst=False,
                 index_col='data'
                )


df.head()

,price,demand,emissions
data,,,
2017-01-01 00:00:00,58.82,24682,14467.2840
2017-01-01 01:00:00,58.23,24046,14173.8295
2017-01-01 02:00:00,51.95,22665,13198.0690
2017-01-01 03:00:00,47.27,21200,12510.7595
2017-01-01 04:00:00,45.49,20056,12203.7315


In [9]:
# Select data from 2021
df = df[int((len(df)*0.8)):]

# Differencing

In [10]:
df.head()

,price,demand,emissions
data,,,
2021-01-01 00:00:00,50.87,25134,6356.6480
2021-01-01 01:00:00,48.19,24179,6269.1480
2021-01-01 02:00:00,44.68,22472,5702.4820
2021-01-01 03:00:00,38.50,20319,5284.9275
2021-01-01 04:00:00,36.80,19040,5218.2340


In [11]:
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)

In [12]:
diff_price = difference(df['price'])
diff_emissions = difference(df['emissions'])
diff_demand = difference(df['demand'])

In [13]:
s1 = pd.Series([0.0])
diff_price = s1.append(diff_price, ignore_index=True)
diff_emissions = s1.append(diff_emissions, ignore_index=True)
diff_demand = s1.append(diff_demand, ignore_index=True)

In [14]:
df['price'] = diff_price.values
df['demand'] = diff_demand.values
df['emissions'] = diff_emissions.values

In [15]:
df.head()

,price,demand,emissions
data,,,
2021-01-01 00:00:00,0.00,0.0,0.0000
2021-01-01 01:00:00,-2.68,-955.0,-87.5000
2021-01-01 02:00:00,-3.51,-1707.0,-566.6660
2021-01-01 03:00:00,-6.18,-2153.0,-417.5545
2021-01-01 04:00:00,-1.70,-1279.0,-66.6935


# Split into training and test

In [16]:
# 80% for training and 20% for test
n = len(df)
train_df = df[0:int(n*0.8)]
test_df = df[int(n*0.8):]

# Normalization/Standarization

In [17]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [18]:
train_df.head()

,price,demand,emissions
data,,,
2021-01-01 00:00:00,-0.001166,0.000057,-0.000507
2021-01-01 01:00:00,-0.121941,-0.737243,-0.163255
2021-01-01 02:00:00,-0.159345,-1.317819,-1.054494
2021-01-01 03:00:00,-0.279669,-1.662149,-0.777150
2021-01-01 04:00:00,-0.077777,-0.987385,-0.124555


# Window (Delete)

In [19]:
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=24, label_width=OUT_STEPS, shift=OUT_STEPS, train_df=train_df, test_df=test_df)
multi_window

Total window size: 48
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
Label column name(s): None

In [20]:
example_window = tf.stack([np.array(test_df[:multi_window.total_window_size]),
                           np.array(test_df[100:100+multi_window.total_window_size]),
                           np.array(test_df[200:200+multi_window.total_window_size])
                          ])

example_inputs, example_labels = multi_window.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

All shapes are: (batch, time, features)
Window shape: (3, 48, 3)
Inputs shape: (3, 24, 3)
Labels shape: (3, 24, 3)


In [21]:
multi_window.example = example_inputs, example_labels

# AR Model

In [22]:
MAX_EPOCHS = 20
OUT_STEPS = 24
num_features=df.shape[1]
patience = 4

In [23]:
multi_val_performance = {}
multi_performance = {}

## Auxiliar methods

In [24]:
class FeedBack(tf.keras.Model):
    def __init__(self, units, out_steps):
        super().__init__()
        self.out_steps = out_steps
        self.units = units

        self.lstm_cell = tf.keras.layers.LSTMCell(units)
        # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
        self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
        self.dense = tf.keras.layers.Dense(num_features)

In [25]:
def warmup(self, inputs):
    # Forma del input => (batch, time, features)
    # Forma de x => (batch, lstm_units)
    x, *state = self.lstm_rnn(inputs)
    
    # Forma de la predicción => (batch, features)
    prediction = self.dense(x)
    return prediction, state

FeedBack.warmup = warmup

In [26]:
def call(self, inputs, training=None):
    # Usar un TensorArray para capturar unrolled outputs automáticamente
    predictions = []
    # Inicializar el estado de LSTM
    prediction, state = self.warmup(inputs)
    
    # Insertar la primera predicción
    predictions.append(prediction)
    
    # Ejecutar el resto de pasos de la predicción
    for n in range(1, self.out_steps):
        # Usar la última predicción como input
        x = prediction
        # Ejecutar un paso de lstm
        x, state = self.lstm_cell(x, states=state, training=training)
        # Convertir la salida de lstm en una predicción
        prediction = self.dense(x)
        # Añadir la predicción al output
        predictions.append(prediction)
    
    # Forma de predictions => (time, batch, features)
    predictions = tf.stack(predictions)
    # Forma de predictions => (batch, time, features)
    predictions = tf.transpose(predictions, [1,0,2])
    
    return predictions

FeedBack.call = call

## Convert the model into a function

In [27]:
def train_test_model(hparams):
    feedback_model = FeedBack(units=hparams[LSTM_UNITS], out_steps=OUT_STEPS)
    
    prediction, state = feedback_model.warmup(multi_window.example[0])
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    
    feedback_model.compile(loss=hparams[LOSS_FUNCTION],
                 optimizer=hparams[OPTIMIZER],
                 metrics=[tf.metrics.RootMeanSquaredError()])
    
    # Creating a Window
    window = WindowGenerator(input_width=hparams[WINDOW_SIZE], label_width=OUT_STEPS, shift=OUT_STEPS, train_df=train_df, test_df=test_df)
    
    # History
    history = feedback_model.fit(window.train, epochs=MAX_EPOCHS, validation_data = window.test, callbacks=[early_stopping])
    
    loss, rmse = feedback_model.evaluate(multi_window.test, verbose=0)
    
    
    return rmse   

## Optimization

In [28]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        rmse = train_test_model(hparams)
        
        tf.summary.scalar(METRIC_RMSE, rmse, step=1)

In [29]:
session_num = 0

for lstm_unit in LSTM_UNITS.domain.values:
    for optimizer in OPTIMIZER.domain.values:
        for loss_function in LOSS_FUNCTION.domain.values:
            for window_size in WINDOW_SIZE.domain.values:
                hparams = {
                    LSTM_UNITS: lstm_unit,
                    OPTIMIZER: optimizer,
                    WINDOW_SIZE: window_size,
                    LOSS_FUNCTION: loss_function
                }
                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run('logs/AR/' + run_name, hparams)
                session_num += 1

--- Starting trial: run-0
{'LSTM_units': 16, 'Optimizer': 'adam', 'WINDOW_SIZE': 12, 'Loss_function': 'MeanSquaredError'}
Epoch 1/20
218/218 [==============================] - 4s 9ms/step - loss: 0.8300 - root_mean_squared_error: 0.9110 - val_loss: 0.9158 - val_root_mean_squared_error: 0.9570
Epoch 2/20
218/218 [==============================] - 1s 5ms/step - loss: 0.5891 - root_mean_squared_error: 0.7675 - val_loss: 0.8860 - val_root_mean_squared_error: 0.9413
Epoch 3/20
218/218 [==============================] - 1s 5ms/step - loss: 0.5663 - root_mean_squared_error: 0.7525 - val_loss: 0.8665 - val_root_mean_squared_error: 0.9309
Epoch 4/20
218/218 [==============================] - 1s 5ms/step - loss: 0.5546 - root_mean_squared_error: 0.7447 - val_loss: 0.8683 - val_root_mean_squared_error: 0.9318
Epoch 5/20
218/218 [==============================] - 1s 5ms/step - loss: 0.5465 - root_mean_squared_error: 0.7392 - val_loss: 0.8584 - val_root_mean_squared_error: 0.9265
Epoch 6/20
218/218

--- Starting trial: run-6
{'LSTM_units': 16, 'Optimizer': 'adam', 'WINDOW_SIZE': 12, 'Loss_function': 'huber'}
Epoch 1/20
218/218 [==============================] - 4s 7ms/step - loss: 0.2966 - root_mean_squared_error: 0.9231 - val_loss: 0.3761 - val_root_mean_squared_error: 1.0309
Epoch 2/20
218/218 [==============================] - 1s 5ms/step - loss: 0.2308 - root_mean_squared_error: 0.8089 - val_loss: 0.3361 - val_root_mean_squared_error: 0.9657
Epoch 3/20
218/218 [==============================] - 1s 5ms/step - loss: 0.2135 - root_mean_squared_error: 0.7775 - val_loss: 0.3240 - val_root_mean_squared_error: 0.9510
Epoch 4/20
218/218 [==============================] - 1s 5ms/step - loss: 0.2079 - root_mean_squared_error: 0.7673 - val_loss: 0.3264 - val_root_mean_squared_error: 0.9535
Epoch 5/20
218/218 [==============================] - 1s 5ms/step - loss: 0.2050 - root_mean_squared_error: 0.7616 - val_loss: 0.3243 - val_root_mean_squared_error: 0.9511
Epoch 6/20
218/218 [=========

Epoch 3/20
218/218 [==============================] - 1s 5ms/step - loss: 0.6425 - root_mean_squared_error: 0.8015 - val_loss: 0.9031 - val_root_mean_squared_error: 0.9503
Epoch 4/20
218/218 [==============================] - 1s 5ms/step - loss: 0.6086 - root_mean_squared_error: 0.7801 - val_loss: 0.8866 - val_root_mean_squared_error: 0.9416
Epoch 5/20
218/218 [==============================] - 1s 5ms/step - loss: 0.5909 - root_mean_squared_error: 0.7687 - val_loss: 0.8812 - val_root_mean_squared_error: 0.9387
Epoch 6/20
218/218 [==============================] - 1s 5ms/step - loss: 0.5795 - root_mean_squared_error: 0.7612 - val_loss: 0.8732 - val_root_mean_squared_error: 0.9344
Epoch 7/20
218/218 [==============================] - 1s 5ms/step - loss: 0.5715 - root_mean_squared_error: 0.7560 - val_loss: 0.8689 - val_root_mean_squared_error: 0.9321
Epoch 8/20
218/218 [==============================] - 1s 5ms/step - loss: 0.5654 - root_mean_squared_error: 0.7519 - val_loss: 0.8661 - val_

Epoch 14/20
217/217 [==============================] - 2s 10ms/step - loss: 0.5261 - root_mean_squared_error: 0.7253 - val_loss: 0.8206 - val_root_mean_squared_error: 0.9059
Epoch 15/20
217/217 [==============================] - 2s 10ms/step - loss: 0.5248 - root_mean_squared_error: 0.7244 - val_loss: 0.8160 - val_root_mean_squared_error: 0.9033
Epoch 16/20
217/217 [==============================] - 2s 10ms/step - loss: 0.5233 - root_mean_squared_error: 0.7234 - val_loss: 0.8161 - val_root_mean_squared_error: 0.9034
Epoch 17/20
217/217 [==============================] - 2s 10ms/step - loss: 0.5217 - root_mean_squared_error: 0.7223 - val_loss: 0.8147 - val_root_mean_squared_error: 0.9026
Epoch 18/20
217/217 [==============================] - 2s 10ms/step - loss: 0.5199 - root_mean_squared_error: 0.7211 - val_loss: 0.8154 - val_root_mean_squared_error: 0.9030
Epoch 19/20
217/217 [==============================] - 2s 10ms/step - loss: 0.5176 - root_mean_squared_error: 0.7194 - val_loss: 0

217/217 [==============================] - 2s 10ms/step - loss: 0.0582 - root_mean_squared_error: 0.7688 - val_loss: 0.0910 - val_root_mean_squared_error: 0.9608
Epoch 15/20
217/217 [==============================] - 2s 10ms/step - loss: 0.0574 - root_mean_squared_error: 0.7643 - val_loss: 0.0916 - val_root_mean_squared_error: 0.9621
Epoch 16/20
217/217 [==============================] - 2s 10ms/step - loss: 0.0567 - root_mean_squared_error: 0.7603 - val_loss: 0.0911 - val_root_mean_squared_error: 0.9572
Epoch 17/20
217/217 [==============================] - 2s 10ms/step - loss: 0.0560 - root_mean_squared_error: 0.7567 - val_loss: 0.0915 - val_root_mean_squared_error: 0.9571
Epoch 18/20
217/217 [==============================] - 2s 10ms/step - loss: 0.0553 - root_mean_squared_error: 0.7534 - val_loss: 0.0907 - val_root_mean_squared_error: 0.9507
Epoch 19/20
217/217 [==============================] - 2s 10ms/step - loss: 0.0547 - root_mean_squared_error: 0.7501 - val_loss: 0.0901 - val_

...

217/217 [==============================] - 14s 64ms/step - loss: 0.1485 - root_mean_squared_error: 0.6168 - val_loss: 0.2587 - val_root_mean_squared_error: 0.8090
Epoch 9/20
217/217 [==============================] - 14s 64ms/step - loss: 0.1414 - root_mean_squared_error: 0.5999 - val_loss: 0.2447 - val_root_mean_squared_error: 0.7820
Epoch 10/20
217/217 [==============================] - 14s 64ms/step - loss: 0.1360 - root_mean_squared_error: 0.5851 - val_loss: 0.2502 - val_root_mean_squared_error: 0.7948
Epoch 11/20
217/217 [==============================] - 14s 64ms/step - loss: 0.1294 - root_mean_squared_error: 0.5641 - val_loss: 0.2486 - val_root_mean_squared_error: 0.7861
Epoch 12/20
217/217 [==============================] - 14s 63ms/step - loss: 0.1221 - root_mean_squared_error: 0.5390 - val_loss: 0.2672 - val_root_mean_squared_error: 0.8223
Epoch 13/20
217/217 [==============================] - 14s 64ms/step - loss: 0.1123 - root_mean_squared_error: 0.5105 - val_loss: 0.2812 -

--- Starting trial: run-137
{'LSTM_units': 128, 'Optimizer': 'sgd', 'WINDOW_SIZE': 48, 'Loss_function': 'MeanSquaredError'}
Epoch 1/20
217/217 [==============================] - 15s 57ms/step - loss: 1.0008 - root_mean_squared_error: 1.0004 - val_loss: 1.5003 - val_root_mean_squared_error: 1.2249
Epoch 2/20
217/217 [==============================] - 13s 59ms/step - loss: 0.9949 - root_mean_squared_error: 0.9975 - val_loss: 1.4905 - val_root_mean_squared_error: 1.2209
Epoch 3/20
217/217 [==============================] - 13s 59ms/step - loss: 0.9892 - root_mean_squared_error: 0.9946 - val_loss: 1.4805 - val_root_mean_squared_error: 1.2167
Epoch 4/20
217/217 [==============================] - 13s 59ms/step - loss: 0.9828 - root_mean_squared_error: 0.9914 - val_loss: 1.4689 - val_root_mean_squared_error: 1.2120
Epoch 5/20
217/217 [==============================] - 13s 59ms/step - loss: 0.9747 - root_mean_squared_error: 0.9873 - val_loss: 1.4537 - val_root_mean_squared_error: 1.2057
Epoch 

Epoch 6/20
218/218 [==============================] - 9s 41ms/step - loss: 0.1136 - root_mean_squared_error: 1.0250 - val_loss: 0.1533 - val_root_mean_squared_error: 1.2470
Epoch 7/20
218/218 [==============================] - 9s 41ms/step - loss: 0.1134 - root_mean_squared_error: 1.0245 - val_loss: 0.1530 - val_root_mean_squared_error: 1.2465
Epoch 8/20
218/218 [==============================] - 9s 41ms/step - loss: 0.1132 - root_mean_squared_error: 1.0240 - val_loss: 0.1527 - val_root_mean_squared_error: 1.2459
Epoch 9/20
218/218 [==============================] - 9s 41ms/step - loss: 0.1130 - root_mean_squared_error: 1.0236 - val_loss: 0.1525 - val_root_mean_squared_error: 1.2454
Epoch 10/20
218/218 [==============================] - 9s 41ms/step - loss: 0.1128 - root_mean_squared_error: 1.0231 - val_loss: 0.1523 - val_root_mean_squared_error: 1.2449
Epoch 11/20
218/218 [==============================] - 9s 41ms/step - loss: 0.1126 - root_mean_squared_error: 1.0227 - val_loss: 0.152

Epoch 12/20
218/218 [==============================] - 7s 33ms/step - loss: 0.3427 - root_mean_squared_error: 0.9918 - val_loss: 0.4928 - val_root_mean_squared_error: 1.2136
Epoch 13/20
218/218 [==============================] - 7s 33ms/step - loss: 0.3418 - root_mean_squared_error: 0.9906 - val_loss: 0.4918 - val_root_mean_squared_error: 1.2121
Epoch 14/20
218/218 [==============================] - 7s 33ms/step - loss: 0.3409 - root_mean_squared_error: 0.9892 - val_loss: 0.4906 - val_root_mean_squared_error: 1.2104
Epoch 15/20
218/218 [==============================] - 7s 33ms/step - loss: 0.3397 - root_mean_squared_error: 0.9875 - val_loss: 0.4892 - val_root_mean_squared_error: 1.2083
Epoch 16/20
218/218 [==============================] - 7s 33ms/step - loss: 0.3382 - root_mean_squared_error: 0.9854 - val_loss: 0.4875 - val_root_mean_squared_error: 1.2057
Epoch 17/20
218/218 [==============================] - 7s 34ms/step - loss: 0.3362 - root_mean_squared_error: 0.9825 - val_loss: 0

Epoch 18/20
217/217 [==============================] - 13s 62ms/step - loss: 0.3383 - root_mean_squared_error: 0.9853 - val_loss: 0.4854 - val_root_mean_squared_error: 1.2046
Epoch 19/20
217/217 [==============================] - 13s 62ms/step - loss: 0.3371 - root_mean_squared_error: 0.9836 - val_loss: 0.4840 - val_root_mean_squared_error: 1.2024
Epoch 20/20
217/217 [==============================] - 14s 62ms/step - loss: 0.3358 - root_mean_squared_error: 0.9815 - val_loss: 0.4822 - val_root_mean_squared_error: 1.1995
